In [ ]:
!pip install ultralytics
!pip install mediapipe
!pip install scikit-learn
!pip install joblib
!pip install pandas

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


ERROR: Operation cancelled by user
^C


In [2]:

!pip install mediapipe

!pip install opencv-python

!pip install scikit-learn

!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 91.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [3]:
# Installing YOLOv5 manually
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [5]:
import os
import cv2
import numpy as np
import mediapipe as mp
import torch
from pathlib import Path
from models.common import DetectMultiBackend
from utils.general import non_max_suppression
from utils.torch_utils import select_device
import joblib
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd  # needed for saving CSV


# Config

yolo_model_path = '/content/best.pt'
mediapipe_model_path = '/content/asl_trained_model Mediapipe Trained.pkl'
test_data_path = '/content/drive/MyDrive/Test_Alphabet'
confidence_threshold = 0.6

letters = [chr(c) for c in range(ord('A'), ord('Z')+1)]


# Loading YOLOv5 Model

device = select_device('')
yolo_model = DetectMultiBackend(yolo_model_path, device=device)
yolo_model.warmup(imgsz=(1, 3, 640, 640))

# Loading MediaPipe Model

clf = joblib.load(mediapipe_model_path)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)


# Utility Functions

def detect_with_yolo(img):
    img_resized = cv2.resize(img, (640, 640))
    img_rgb = img_resized[..., ::-1].transpose((2, 0, 1))
    img_rgb = np.ascontiguousarray(img_rgb)
    img_tensor = torch.from_numpy(img_rgb).to(device).float() / 255.0
    if img_tensor.ndimension() == 3:
        img_tensor = img_tensor.unsqueeze(0)

    pred = yolo_model(img_tensor, augment=False, visualize=False)
    pred = non_max_suppression(pred, conf_thres=confidence_threshold)[0]

    if pred is not None and len(pred):
        cls_id = int(pred[0][5].item())
        detected_class = yolo_model.names[cls_id]
        return detected_class
    else:
        return None

def detect_with_mediapipe(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    if results.multi_hand_landmarks:
        landmarks = []
        for lm in results.multi_hand_landmarks[0].landmark:
            landmarks.extend([lm.x, lm.y, lm.z])
        return clf.predict([landmarks])[0]
    else:
        return None


# Evaluation Loop

y_true = []
y_pred_combined = []
y_pred_yolo = []
y_pred_mediapipe = []

for label in letters:
    folder = os.path.join(test_data_path, label)
    if not os.path.exists(folder):
        print(f"Skipping '{label}' (folder not found)")
        continue

    images = os.listdir(folder)
    for img_name in tqdm(images, desc=f"Processing {label}"):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        pred_yolo = detect_with_yolo(img) or 'None'
        pred_mediapipe = detect_with_mediapipe(img) or 'None'

        # Rule for Combined Prediction (late fusion)
        if pred_yolo == pred_mediapipe and pred_yolo != 'None':
            combined = pred_yolo  # Both agree
        elif pred_yolo != 'None' and pred_mediapipe == 'None':
            combined = pred_yolo  # Only YOLO confident
        elif pred_mediapipe != 'None' and pred_yolo == 'None':
            combined = pred_mediapipe  # Only MediaPipe confident
        elif pred_yolo != pred_mediapipe and pred_yolo != 'None' and pred_mediapipe != 'None':
            combined = 'Mismatch'  # Both predict different letters
        else:
            combined = 'None'  # Both failed

        y_true.append(label)
        y_pred_combined.append(combined)
        y_pred_yolo.append(pred_yolo)
        y_pred_mediapipe.append(pred_mediapipe)


# Report Results

def plot_confusion(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred, labels=letters + ['None', 'Mismatch'])
    plt.figure(figsize=(16, 12))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=letters + ['None', 'Mismatch'],
                yticklabels=letters + ['None', 'Mismatch'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.show()

def print_report(name, y_pred):
    print(f"\n=== {name} Classification Report ===")
    print(classification_report(y_true, y_pred, labels=letters + ['None', 'Mismatch']))
    print(f"{name} Accuracy: {accuracy_score(y_true, y_pred) * 100:.2f}%")

print_report("YOLOv5 Only", y_pred_yolo)
print_report("MediaPipe Only", y_pred_mediapipe)
print_report("Combined Model", y_pred_combined)

plot_confusion(y_true, y_pred_combined, "Combined Model Confusion Matrix")


pd.DataFrame({
    'True': y_true,
    'YOLOv5': y_pred_yolo,
    'MediaPipe': y_pred_mediapipe,
    'Combined': y_pred_combined
}).to_csv('/content/combined_results.csv', index=False)
print(" Combined results saved to '/content/combined_results.csv'")




# Overall classification accuracy of the late-fusion
acc_combined = accuracy_score(y_true, y_pred_combined)
print("\n=== Late Fusion (Combined) — Overall Accuracy ===")
print(f"Accuracy: {acc_combined * 100:.2f}%")

# Macro-averaged Precision, Recall, F1 over the 26 ASL letters
prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true, y_pred_combined, labels=letters, average='macro', zero_division=0
)
print("\n=== Late Fusion (Combined) — Macro-Averaged Metrics (A–Z only) ===")
print(f"Precision (Macro): {prec_macro:.4f}")
print(f"Recall (Macro):    {rec_macro:.4f}")
print(f"F1-Score (Macro):  {f1_macro:.4f}")

# Confusion matrix for late fusion
cm_letters_only = confusion_matrix(y_true, y_pred_combined, labels=letters)
plt.figure(figsize=(16, 12))
sns.heatmap(
    cm_letters_only, annot=True, fmt='d', cmap='Blues',
    xticklabels=letters, yticklabels=letters
)
plt.xlabel('Predicted (Late Fusion, A–Z)')
plt.ylabel('True (A–Z)')
plt.title('Late Fusion (Combined) Confusion Matrix — Letters Only')
plt.show()


YOLOv5 🚀 v7.0-425-g85acef3a Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7313943 parameters, 0 gradients
Processing Z: 100%|██████████| 100/100 [01:10<00:00,  1.41it/s]



=== YOLOv5 Only Classification Report ===
              precision    recall  f1-score   support

           A       1.00      0.99      0.99       100
           B       1.00      0.86      0.92       100
           C       1.00      0.94      0.97       100
           D       0.96      0.71      0.82       100
           E       1.00      0.98      0.99       100
           F       1.00      0.94      0.97       100
           G       0.98      0.64      0.78       100
           H       1.00      0.78      0.88       100
           I       0.99      0.84      0.91       100
           J       1.00      0.94      0.97       100
           K       1.00      0.72      0.84       100
           L       1.00      0.91      0.95       100
           M       1.00      0.83      0.91       100
           N       1.00      0.90      0.95       100
           O       0.99      0.98      0.98       100
           P       0.99      0.96      0.97       100
           Q       1.00      0.92     

In [6]:
plt.savefig('/content/confusion_combined.png', dpi=300, bbox_inches='tight')